In [1]:
!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat


<class 'OSError'>: Not available

In [2]:
from scipy.io import loadmat
import numpy as np

# Load the data
train_data = loadmat('train_32x32.mat')
test_data = loadmat('test_32x32.mat')

# Preprocess the images (normalize and reshape)
x_train = np.transpose(train_data['X'], (3, 0, 1, 2)).astype('float32') / 255.0
y_train = train_data['y'].flatten()
x_test = np.transpose(test_data['X'], (3, 0, 1, 2)).astype('float32') / 255.0
y_test = test_data['y'].flatten()

# Display dataset shapes
print(f'Training data shape: {x_train.shape}')
print(f'Test data shape: {x_test.shape}')


<class 'FileNotFoundError'>: [Errno 44] No such file or directory: 'train_32x32.mat'

In [3]:
from tensorflow.keras import layers, models

def build_encoder(latent_dim):
    # Define the encoder model
    encoder_input = layers.Input(shape=(32, 32, 3))
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_input)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    z_mean = layers.Dense(latent_dim)(x)
    z_log_var = layers.Dense(latent_dim)(x)

    # Return encoder model and latent representations
    encoder = models.Model(encoder_input, [z_mean, z_log_var])
    return encoder

# Example usage: build the encoder with latent dimension of 2
encoder = build_encoder(latent_dim=2)
encoder.summary()


<class 'ModuleNotFoundError'>: No module named 'tensorflow'

In [4]:
def build_decoder(latent_dim):
    decoder_input = layers.Input(shape=(latent_dim,))
    x = layers.Dense(8*8*64, activation='relu')(decoder_input)
    x = layers.Reshape((8, 8, 64))(x)
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    decoder_output = layers.Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)
    
    decoder = models.Model(decoder_input, decoder_output)
    return decoder

def build_vae(latent_dim):
    # Encoder
    encoder = build_encoder(latent_dim)
    z_mean, z_log_var = encoder.output
    
    # Latent space sampling
    def sample_z(args):
        z_mean, z_log_var = args
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    
    z = layers.Lambda(sample_z)([z_mean, z_log_var])

    # Decoder
    decoder = build_decoder(latent_dim)
    vae_output = decoder(z)

    # VAE model
    vae = models.Model(encoder.input, vae_output)
    return vae

vae = build_vae(latent_dim=2)
vae.summary()


<class 'NameError'>: name 'build_encoder' is not defined

In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model

<class 'ModuleNotFoundError'>: No module named 'tensorflow'

In [6]:
# ModelCheckpoint to save the model only when validation loss improves
checkpoint_callback = ModelCheckpoint(
    'vae_model.h5',  # Path where the model will be saved
    save_best_only=True,  # Save only the best model
    monitor='val_loss',  # Monitor validation loss for improvements
    mode='min',  # Save when the validation loss decreases
    verbose=1  # Output messages when saving the model
)

# EarlyStopping to stop training after 10 epochs of no improvement
early_stopping_callback = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=10,  # Stop after 10 epochs with no improvement
    verbose=1  # Print messages when stopping
)


<class 'NameError'>: name 'ModelCheckpoint' is not defined

In [7]:
history = vae.fit(
    train_data,  # Training data
    epochs=100,  # Number of epochs to train
    batch_size=32,  # Batch size
    validation_data=(val_data, val_data),  # Validation data
    callbacks=[checkpoint_callback, early_stopping_callback]  # Apply the callbacks
)


<class 'NameError'>: name 'vae' is not defined

In [8]:
# Load the saved model with custom loss function
vae = load_model('vae_model.h5', custom_objects={'vae_loss': vae_loss})

# Continue training the loaded model
history = vae.fit(
    train_data,
    epochs=additional_epochs,  # Number of additional epochs to train
    batch_size=32,  # Maintain the same batch size or adjust as needed
    validation_data=(val_data, val_data),  # Validation data
    callbacks=[checkpoint_callback, early_stopping_callback]  # Use callbacks again
)


<class 'NameError'>: name 'load_model' is not defined

In [9]:
history = vae.fit(
    train_data,
    epochs=50,  # Train for 50 epochs
    batch_size=64,  # Batch size of 64
    validation_data=(val_data, val_data),  # Validation data
    callbacks=[checkpoint_callback, early_stopping_callback]  # Apply the callbacks
)


<class 'NameError'>: name 'vae' is not defined